In [ ]:
"""
There is the customer_master.xlsx file as a definition table for data preparation. 
It helps to exlude wrong inforamtion in an internal data source which cannot be corrected.

columns: ['Customer', 'Tier', 'Accounts', 'Company', 'the_newest_OPCO',
       'Customer Name', 'ONE_NAME', 'Indirect/Direct', 'Channel3', 'New Type4']

There is a budget file which budget information for two fiscal years. 

columns: ['sold_to_customer', 'sold_to_customer_n', 'Country', 'Country Name',
       'City', 'Type', 'OI Bgt FY21', 'OI Bgt FY22', 'Comments']

Main goal of this code to combine these two files in one which would be used as one of 
Tableau data sources.

"""

In [53]:
import pandas as pd
import numpy as np

#budget df preparation
budget_df = pd.read_excel('data_files/budget_file.xlsx', sheet_name='Budget')
budget_df = budget_df.loc[:,['sold_to_customer','sold_to_customer_n','Type', 'OI Bgt FY21', 'OI Bgt FY22']]
budget_df.sold_to_customer = budget_df.sold_to_customer.astype('str') # the internal database has ID beginning with 0 in str format
budget_df.sold_to_customer = "0" + budget_df.sold_to_customer

#customer master df preparation
customer_df = pd.read_excel('data_files/customer_master.xlsx', sheet_name='data')
customer_df = customer_df.loc[:,['Customer', 'Tier', 'the_newest_OPCO',
       'Customer Name', 'ONE_NAME', 'Indirect/Direct', 'Channel3', 'New Type4']]
customer_df = customer_df.rename(columns={'Customer':'sold_to_customer','the_newest_OPCO':'OPCO', 'Channel3':'Channel', 'New Type4':'Type'})
customer_df.sold_to_customer = customer_df.sold_to_customer.astype('str') # the internal database has ID beginning with 0 in str format
customer_df.sold_to_customer = "0" + customer_df.sold_to_customer
customer_df = customer_df.set_index('sold_to_customer')

In [54]:
df = budget_df.join(customer_df, on='sold_to_customer', how = 'left', lsuffix = 'l_', rsuffix = 'r_')

In [47]:
writer = pd.ExcelWriter('budget_file.xlsx', engine='xlsxwriter')
df.to_excel(writer, sheet_name='budget_by_month')
writer.save()

In [55]:
df2=customer_df.dropna(subset=['ONE_NAME'])